In [ ]:
import tensorflow as tf
print(tf.__version__)

# downloading tensorflow docs
!pip install -q git+https://github.com/tensorflow/docs
    
# importing modules
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop, Adam, SGD
from sklearn.model_selection import train_test_split
from keras import regularizers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Importing Dataset
dataset = pd.read_csv('Dataset 10 day.csv')
# Deleting Null Values
dataset.dropna(inplace = True)
# Encoding the Region Column
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(dataset.region)
dataset['region'] = le.transform(dataset.region)

cln = [c for c in dataset.columns if c not in ['region', 'NewCase Moving Av 10 day FUT']]
# normalising variables without new cases and region variables
x = dataset.loc[:, cln] 
x_norm = (x - x.mean()) / (x.max() - x.min())

# creating a new DF and new cases and region
df = pd.DataFrame(x_norm)
df['NC'] = dataset['NewCase Moving Av 10 day FUT'] 
df.insert(0, 'region', dataset['region'])

X = df.iloc[:, :-1]
y = df.iloc[:, -1]

# splitting the data to training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2,
                                                    random_state=0)

# designing the network

model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

optimizer = RMSprop(0.001)

model.compile(loss='mse',
              optimizer=optimizer,
              metrics=['mae', 'mse']) # Mean Absolute Error (mae), Mean Squared Error (mse) 

model.summary()

# validation is taken from the training set to check the accuracy as the model is learning  

EPOCHS = 1000

history = model.fit(
  X_train, y_train,
  epochs=EPOCHS, validation_split=0.2, verbose=0,
  callbacks=[tfdocs.modeling.EpochDots()])

# geting the info of each epoch for later use
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

# Plotting the loss using the mean absolute error
plotter = tfdocs.plots.HistoryPlotter(smoothing_std=3)
plotter.plot({'Basic': history}, metric="mae")
plt.ylim([100, 1400])
plt.ylabel('MAE [covid-19 case]')
# Plotting the loss using the mean standard error
plotter.plot({'Basic': history}, metric="mse")
plt.ylim([0, 10000000])
plt.ylabel('MSE [covid-19 case^2]')

# evaluting the model using the test set
loss, mae, mse = model.evaluate(X_test, y_test, verbose=2)
print("Testing set Mean Abs Error: {:5.2f} number of covid-19 cases".format(mae))
print("Testing set Mean Squared Error: {:5.2f} number of covid-19 cases".format(mse))

# making predictions using the data in the test set

test_predictions = model.predict(X_test).flatten()

a = plt.axes(aspect='equal')
plt.scatter(y_test, test_predictions)
plt.xlabel('True Values [cases]')
plt.ylabel('Predictions [cases]')
lims = [0, 5000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)

# checking the error distribution

error = test_predictions - y_test
plt.hist(error, bins = 100)
plt.xlabel("Prediction Error [cases]")
_ = plt.ylabel("Count")

# calculating R Squared Values
from sklearn.metrics import r2_score
r2 = r2_score(y_test,test_predictions)
Adj_r2 = 1-((1-r2)*(len(y_test)-1)/(len(y_test)-13-1))
Adj_r2

# viewing y test vs y pred values
y_test = y_test.to_numpy()
np.set_printoptions(precision=2)
Results_data = np.concatenate((test_predictions.reshape(len(test_predictions),1), y_test.reshape(len(y_test),1)),1)
CoL = ("y_pred","y_test")
pd.DataFrame(data = Results_data, columns = CoL)

2.3.0
